# BERT word embeddings + various classification algorithms

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import gensim

import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import re, string
import nltk
from nltk.corpus import stopwords
import pickle
from tabulate import tabulate

import xgboost as xgb

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-04-07 23:56:59.968526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
fake = pd.read_csv('../../data/Fake.csv')
true = pd.read_csv('../../data/True.csv')

fake["label"] = 0
true["label"] = 1

df = pd.concat([fake, true], ignore_index = True)

df['text'] = df['title'] + " " + df['text']
df.drop(columns=['title', 'date', 'subject'], inplace = True)

In [3]:
%%time
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)


def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text = re.sub('\[[^]]*\]', '', text)
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
    
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

df['text']=df['text'].apply(denoise_text)

[nltk_data] Downloading package stopwords to /home/szymon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/szymon/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


CPU times: user 8.65 s, sys: 73.9 ms, total: 8.72 s
Wall time: 9.05 s


---

Reduce dataset for testing purposes

In [4]:
# df_original = df.copy()
# df = df.sample(frac=1).reset_index(drop=True)[:1000]

---

# Embedding

In [5]:
redo_embedding = True # recalculate embeddings
fast = True # True if use reduced dataset (1000 obs) vs. False if full dataset (40000 obs)

In [8]:
%%time
# Load data

if fast:
    df_original = df.copy()
    df = df.sample(frac=1).reset_index(drop=True)[:1000]

if redo_embedding:
    X = df['text'].tolist()
    y = df['label'].tolist()

    with open("X", "wb") as fp:
      pickle.dump(X, fp)
    with open("y", "wb") as fp:
      pickle.dump(y, fp)
elif fast:
    with open("../../data/small/X", "rb") as fp:
      X = pickle.load(fp)
    with open("../../data/small/y", "rb") as fp:
      y = pickle.load(fp)
else:
    with open("../../data/X", "rb") as fp:
      X = pickle.load(fp)
    with open("../../data/y", "rb") as fp:
      y = pickle.load(fp)
    
if redo_embedding:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    with open("X_train", "wb") as fp:
      pickle.dump(X_train, fp)
    with open("X_test", "wb") as fp:
      pickle.dump(X_test, fp)
    with open("y_train", "wb") as fp:
      pickle.dump(y_train, fp)
    with open("y_test", "wb") as fp:
      pickle.dump(y_test, fp)
elif fast:
    with open("../../data/small/X_train", "rb") as fp:
      X_train = pickle.load(fp)
    with open("../../data/small/X_test", "rb") as fp:
      X_test = pickle.load(fp)   
    with open("../../data/small/y_train", "rb") as fp:
      y_train = pickle.load(fp)
    with open("../../data/small/y_test", "rb") as fp:
      y_test = pickle.load(fp)
else:
    with open("../../data/X_train", "rb") as fp:
      X_train = pickle.load(fp)
    with open("../../data/X_test", "rb") as fp:
      X_test = pickle.load(fp)   
    with open("../../data/y_train", "rb") as fp:
      y_train = pickle.load(fp)
    with open("../../data/y_test", "rb") as fp:
      y_test = pickle.load(fp)

CPU times: user 16.9 ms, sys: 52.2 ms, total: 69 ms
Wall time: 68.5 ms


## BERT Embedding

In [9]:
if redo_embedding:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    bert = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    bert.to(device)

    def _get_bert_embedding(text):
        input_ids = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512)
        input_ids = np.array(input_ids)
        input_ids = np.expand_dims(input_ids, axis=0)
        input_ids = torch.tensor(input_ids).to(device)

        with torch.no_grad():
            outputs = bert(input_ids)
            last_hidden_state = outputs.last_hidden_state
            last_hidden_state = last_hidden_state[:, 0, :].cpu().numpy()

        return last_hidden_state

    print("TRAIN")
    X_train_embeddings = []
    for text in tqdm(X_train):
        embedding = _get_bert_embedding(text)
        X_train_embeddings.append(embedding)
    X_train_embeddings = np.array(X_train_embeddings)
    X_train_embeddings_bert = np.squeeze(X_train_embeddings, axis=1)

    print("TEST")
    X_test_embeddings = []
    for text in tqdm(X_test):
        embedding = _get_bert_embedding(text)
        X_test_embeddings.append(embedding)
    X_test_embeddings = np.array(X_test_embeddings)
    X_test_embeddings_bert = np.squeeze(X_test_embeddings, axis=1)
    
    if fast:
        pd.DataFrame(X_train_embeddings_bert).to_csv("../../data/small/embeddings/X_train_embeddings_bert_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_bert).to_csv("../../data/small/embeddings/X_test_embeddings_bert_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
    else:
        pd.DataFrame(X_train_embeddings_bert).to_csv("../../data/embeddings/X_train_embeddings_bert_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_bert).to_csv("../../data/embeddings/X_test_embeddings_bert_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
    
elif fast:
    X_train_embeddings_bert = pd.read_csv('../../data/small/embeddings/X_train_embeddings_bert.csv', sep=',', header=None).values
    X_test_embeddings_bert = pd.read_csv('../../data/small/embeddings/X_test_embeddings_bert.csv', sep=',', header=None).values
else:
    X_train_embeddings_bert = pd.read_csv('../../data/bert/X_train_embeddings_bert.csv', sep=',', header=None).values
    X_test_embeddings_bert = pd.read_csv('../../data/bert/X_test_embeddings_bert.csv', sep=',', header=None).values

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TRAIN


100%|██████████| 800/800 [00:28<00:00, 28.05it/s]


TEST


100%|██████████| 200/200 [00:06<00:00, 29.64it/s]


## GloVe Embedding

In [11]:
if redo_embedding:

    def load_glove_embeddings(filename):
        embeddings_index = {}
        with open(filename) as f:
            for line in tqdm(f):
                values = line.split()
                word = values[0]
                if len(values[1:]) == 300:
                    coefs = np.asarray(values[1:], dtype='float32')
                    embeddings_index[word] = coefs
        return embeddings_index

    glove_embeddings = load_glove_embeddings('../../glove/glove.840B.300d.txt')

    def text_to_glove_embeddings(text, embeddings_index, embedding_dim):
        embeddings = []
        for sentence in text:
            sentence_embeddings = []
            for word in sentence.split():
                if word in embeddings_index:
                    sentence_embeddings.append(embeddings_index[word])
            if len(sentence_embeddings) > 0:
                embeddings.append(np.mean(sentence_embeddings, axis=0))
            else:
                embeddings.append(np.zeros(embedding_dim))
        return np.array(embeddings)

    X_train_embeddings_glove = text_to_glove_embeddings(X_train, glove_embeddings, embedding_dim=300)
    X_test_embeddings_glove = text_to_glove_embeddings(X_test, glove_embeddings, embedding_dim=300)
    
    if fast:
        pd.DataFrame(X_train_embeddings_glove).to_csv("../../data/small/embeddings/X_train_embeddings_glove_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_glove).to_csv("../../data/small/embeddings/X_test_embeddings_glove_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
    else:
        pd.DataFrame(X_train_embeddings_glove).to_csv("../../data/embeddings/X_train_embeddings_glove_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_glove).to_csv("../../data/embeddings/X_test_embeddings_glove_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)

elif fast:
    X_train_embeddings_glove = pd.read_csv('../../data/small/embeddings/X_train_embeddings_glove.csv', sep=',', header=None).values
    X_test_embeddings_glove = pd.read_csv('../../data/small/embeddings/X_test_embeddings_glove.csv', sep=',', header=None).values

else:
    X_train_embeddings_glove = pd.read_csv('../../data/embeddings/X_train_embeddings_glove.csv', sep=',', header=None).values
    X_test_embeddings_glove = pd.read_csv('../../data/embeddings/X_test_embeddings_glove.csv', sep=',', header=None).values

## Word2Vec

In [12]:
if redo_embedding:
    model = gensim.models.KeyedVectors.load_word2vec_format('../../word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

def get_word2vec_embeddings(text):
    embeddings = []
    for sentence in tqdm(text):
        tokens = sentence.split()
        doc_vecs = [model[token] for token in tokens if token in model.key_to_index]
        if len(doc_vecs) > 0:
            doc_vec = np.mean(doc_vecs, axis=0)
            embeddings.append(doc_vec)
    return np.array(embeddings)


if redo_embedding:
    # noinspection PyUnboundLocalVariable
    X_train_embeddings_word2vec = get_word2vec_embeddings(X_train)
    X_test_embeddings_word2vec = get_word2vec_embeddings(X_test)

    if fast:
        pd.DataFrame(X_train_embeddings_word2vec).to_csv("../../data/small/embeddings/X_train_embeddings_word2vec_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_word2vec).to_csv("../../data/small/embeddings/X_test_embeddings_word2vec_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
    else:
        pd.DataFrame(X_train_embeddings_word2vec).to_csv("../../data/embeddings/X_train_embeddings_word2vec_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)
        pd.DataFrame(X_test_embeddings_word2vec).to_csv("../../data/embeddings/X_test_embeddings_word2vec_"+time.strftime("%Y%m%d-%H%M%S")+".csv", index=False, header=False)

elif fast:
    X_train_embeddings_word2vec = pd.read_csv('../../data/small/embeddings/X_train_embeddings_word2vec.csv', sep=',', header=None).values
    X_test_embeddings_word2vec = pd.read_csv('../../data/small/embeddings/X_test_embeddings_word2vec.csv', sep=',', header=None).values

else:
    X_train_embeddings_word2vec = pd.read_csv('../../data/embeddings/X_train_embeddings_word2vec.csv', sep=',', header=None).values
    X_test_embeddings_word2vec = pd.read_csv('../../data/embeddings/X_test_embeddings_word2vec.csv', sep=',', header=None).values

100%|██████████| 200/200 [00:00<00:00, 2883.98it/s]


# Classification

## Wrappers

### KNN

In [13]:
class KNNClassifier:
    def __init__(self, n_neighbors=2, weights='uniform', metric='minkowski'):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.metric = metric
        self.model = None

    def fit(self, X_train, y_train):
        self.model = KNeighborsClassifier(n_neighbors=self.n_neighbors, weights=self.weights, metric=self.metric)
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        if self.model is None:
          raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.model.predict(X_test)
        return y_pred

    def randomized_search(self, X_train, y_train, param_distributions, cv=5, n_iter=20):
        self.model = KNeighborsClassifier()
        random_search = RandomizedSearchCV(self.model, param_distributions=param_distributions, cv=cv, n_iter=n_iter, verbose=3)
        random_search.fit(X_train, y_train)

        self.n_neighbors = random_search.best_params_['n_neighbors']
        self.weights = random_search.best_params_['weights']
        self.metric = random_search.best_params_['metric']

        self.model = KNeighborsClassifier(n_neighbors=self.n_neighbors, weights=self.weights, metric=self.metric)

    def evaluate(self, X_test, y_test):
        if self.model is None:
              raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = round(accuracy_score(y_test, y_pred)*100,1)
        f1 = round(f1_score(y_test, y_pred)*100,1)

        print('\nConfusion matrix\n', cm, '\n')
        print('Accuracy:', accuracy, '\n')
        print('F1 Score:', f1, '\n')

        return cm, accuracy, f1

### XGBoost

In [14]:
class XGBoostClassifier:
    def __init__(self, learning_rate=0.1, max_depth=5, min_child_weight=1, subsample=0.5, colsample_bytree=0.5, n_estimators=100, objective='req:squarederror'):
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.n_estimators = n_estimators
        self.objective = objective
        self.model = None

    def fit(self, X_train, y_train):
        self.model = xgb.XGBClassifier(learning_rate=self.learning_rate, max_depth=self.max_depth, min_child_weight=self.min_child_weight, subsample=self.subsample, colsample_bytree=self.colsample_bytree, n_estimators=self.n_estimators, objective=self.objective)
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        if self.model is None:
          raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.model.predict(X_test)

        return y_pred

    def randomized_search(self, X_train, y_train, param_distributions, cv=5, n_iter=20):
        self.model = xgb.XGBClassifier()
        random_search = RandomizedSearchCV(self.model, param_distributions=param_distributions, cv=cv, n_iter=n_iter, verbose=3)
        random_search.fit(X_train, y_train)

        self.learning_rate = random_search.best_params_['learning_rate']
        self.max_depth = random_search.best_params_['max_depth']
        self.min_child_weight = random_search.best_params_['min_child_weight']
        self.subsample = random_search.best_params_['subsample']
        self.colsample_bytree = random_search.best_params_['colsample_bytree']
        self.n_estimators = random_search.best_params_['n_estimators']
        self.objective = random_search.best_params_['objective']

        self.model = xgb.XGBClassifier(learning_rate=self.learning_rate, max_depth=self.max_depth, min_child_weight=self.min_child_weight, subsample=self.subsample, colsample_bytree=self.colsample_bytree, n_estimators=self.n_estimators, objective=self.objective)

    def evaluate(self, X_test, y_test):
        if self.model is None:
              raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = round(accuracy_score(y_test, y_pred)*100,1)
        f1 = round(f1_score(y_test, y_pred)*100,1)

        print('\nConfusion matrix\n', cm, '\n')
        print('Accuracy:', accuracy, '\n')
        print('F1 Score:', f1, '\n')

        return cm, accuracy, f1

### Random Forest

In [15]:
class RFClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth='none', bootstrap=True):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.bootstrap = bootstrap
        self.model = None

    def fit(self, X_train, y_train):
        self.model = RandomForestClassifier(n_estimators=self.n_estimators, max_features=self.max_features, max_depth=self.max_depth, bootstrap=self.bootstrap, verbose=True)
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        if self.model is None:
          raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.model.predict(X_test)

        return y_pred

    def randomized_search(self, X_train, y_train, param_distributions, cv=5, n_iter=20):
        self.model = RandomForestClassifier()
        random_search = RandomizedSearchCV(self.model, param_distributions=param_distributions, cv=cv, n_iter=n_iter, verbose=3)
        random_search.fit(X_train, y_train)

        self.n_estimators = random_search.best_params_['n_estimators']
        self.max_features = random_search.best_params_['max_features']
        self.max_depth = random_search.best_params_['max_depth']
        self.bootstrap = random_search.best_params_['bootstrap']

        self.model = RandomForestClassifier(n_estimators=self.n_estimators, max_features=self.max_features, max_depth=self.max_depth, bootstrap=self.bootstrap)

    def evaluate(self, X_test, y_test):
        if self.model is None:
              raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = round(accuracy_score(y_test, y_pred)*100,1)
        f1 = round(f1_score(y_test, y_pred)*100,1)

        print('\nConfusion matrix\n', cm, '\n')
        print('Accuracy:', accuracy, '\n')
        print('F1 Score:', f1, '\n')

        return cm, accuracy, f1

### SVC

In [16]:
class SVClassifier:
    def __init__(self, C = 1, kernel='linear', gamma = 0.2):
        self.C = C
        self.kernel = kernel
        self.gamma = gamma
        self.model = None

    def fit(self, X_train, y_train):
        self.model = svm.SVC(C=self.C, kernel=self.kernel, gamma=self.gamma, verbose=True)
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        if self.model is None:
          raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.model.predict(X_test)

        return y_pred

    def randomized_search(self, X_train, y_train, param_distributions, cv=5, n_iter=20):
        self.model = svm.SVC()
        random_search = RandomizedSearchCV(self.model, param_distributions=param_distributions, cv=cv, n_iter=n_iter, verbose=3)
        random_search.fit(X_train, y_train)

        self.C = random_search.best_params_['C']
        self.kernel = random_search.best_params_['kernel']
        self.gamma = random_search.best_params_['gamma']

        self.model = svm.SVC(C=self.C, kernel=self.kernel, gamma=self.gamma)

    def evaluate(self, X_test, y_test):
        if self.model is None:
              raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = round(accuracy_score(y_test, y_pred)*100,1)
        f1 = round(f1_score(y_test, y_pred)*100,1)

        print('\nConfusion matrix\n', cm, '\n')
        print('Accuracy:', accuracy, '\n')
        print('F1 Score:', f1, '\n')

        return cm, accuracy, f1

### Logistic Regression

In [17]:
class LRClassifier:
    def __init__(self, penalty = 'l2', solver = 'libinear', C = 0.5):
        self.penalty = penalty
        self.solver = solver
        self.C = C
        self.model = None

    def fit(self, X_train, y_train):
        self.model = LogisticRegression(penalty=self.penalty, solver=self.solver, C=self.C)
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        if self.model is None:
          raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.model.predict(X_test)

        return y_pred

    def randomized_search(self, X_train, y_train, param_distributions, cv=5, n_iter=20):
        self.model = LogisticRegression()
        random_search = RandomizedSearchCV(self.model, param_distributions=param_distributions, cv=cv, n_iter=n_iter, verbose=3)
        random_search.fit(X_train, y_train)

        self.penalty = random_search.best_params_['penalty']
        self.solver = random_search.best_params_['solver']
        self.C = random_search.best_params_['C']

        self.model = LogisticRegression(penalty=self.penalty, solver=self.solver, C=self.C)

    def evaluate(self, X_test, y_test):
        if self.model is None:
              raise ValueError("The model has not been trained yet. Please call 'fit' first.")
        y_pred = self.predict(X_test)

        cm = confusion_matrix(y_test, y_pred)
        accuracy = round(accuracy_score(y_test, y_pred)*100,1)
        f1 = round(f1_score(y_test, y_pred)*100,1)

        print('\nConfusion matrix\n', cm, '\n')
        print('Accuracy:', accuracy, '\n')
        print('F1 Score:', f1, '\n')

        return cm, accuracy, f1

## BERT embeddings

### BERT + KNN

In [18]:
# Instantiate classifier
classifier = KNNClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9],
    'weights' : ['uniform','distance'],
    'metric' : ['minkowski','euclidean','manhattan']
}
classifier.randomized_search(X_train_embeddings_bert, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_bert, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_knn_bert_train, accuracy_knn_bert_train, f1_knn_bert_train = classifier.evaluate(X_train_embeddings_bert, y_train)

print('\n----- TEST -----')
cm_knn_bert_test, accuracy_knn_bert_test, f1_knn_bert_test = classifier.evaluate(X_test_embeddings_bert, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.969 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.944 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.919 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.969 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.944 total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=5, weights=uniform;, score=0.975 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=5, weights=uniform;, score=0.969 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=5, weights=uniform;, score=0.938 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=5, weights=uniform;, score=0.969 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=5, weights=uniform;, score=0.931 to

### BERT + XGBoost

In [19]:
# Instantiate classifier
classifier = XGBoostClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'learning_rate': [0.01, 0.1, 0.3, 0.5],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7],
    'n_estimators': [50, 100, 200, 500],
    'objective': ['reg:squarederror']
}
classifier.randomized_search(X_train_embeddings_bert, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_bert, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_xgb_bert_train, accuracy_xgb_bert_train, f1_xgb_bert_train = classifier.evaluate(X_train_embeddings_bert, y_train)

print('\n----- TEST -----')
cm_xgb_bert_test, accuracy_xgb_bert_test, f1_xgb_bert_test = classifier.evaluate(X_test_embeddings_bert, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END colsample_bytree=0.7, learning_rate=0.5, max_depth=3, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.7;, score=0.938 total time=   1.8s
[CV 2/5] END colsample_bytree=0.7, learning_rate=0.5, max_depth=3, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.7;, score=0.919 total time=   2.0s
[CV 3/5] END colsample_bytree=0.7, learning_rate=0.5, max_depth=3, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.7;, score=0.906 total time=   1.3s
[CV 4/5] END colsample_bytree=0.7, learning_rate=0.5, max_depth=3, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.7;, score=0.919 total time=   1.4s
[CV 5/5] END colsample_bytree=0.7, learning_rate=0.5, max_depth=3, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.7;, score=0.887 total time=   1.2s
[CV 1/5] END colsample_bytree=0.7, lea

### BERT + Random Forest

In [20]:
# Instantiate classifier
classifier = RFClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_estimators': [10, 25], 
    'max_features': [5, 10],
    'max_depth': [10, 50, None], 
    'bootstrap': [True, False]
}
classifier.randomized_search(X_train_embeddings_bert, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_bert, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_rf_bert_train, accuracy_rf_bert_train, f1_rf_bert_train = classifier.evaluate(X_train_embeddings_bert, y_train)

print('\n----- TEST -----')
cm_rf_bert_test, accuracy_rf_bert_test, f1_rf_bert_test = classifier.evaluate(X_test_embeddings_bert, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END bootstrap=False, max_depth=None, max_features=5, n_estimators=25;, score=0.975 total time=   0.1s
[CV 2/5] END bootstrap=False, max_depth=None, max_features=5, n_estimators=25;, score=0.950 total time=   0.1s
[CV 3/5] END bootstrap=False, max_depth=None, max_features=5, n_estimators=25;, score=0.925 total time=   0.1s
[CV 4/5] END bootstrap=False, max_depth=None, max_features=5, n_estimators=25;, score=0.956 total time=   0.1s
[CV 5/5] END bootstrap=False, max_depth=None, max_features=5, n_estimators=25;, score=0.969 total time=   0.1s
[CV 1/5] END bootstrap=True, max_depth=50, max_features=10, n_estimators=10;, score=0.925 total time=   0.0s
[CV 2/5] END bootstrap=True, max_depth=50, max_features=10, n_estimators=10;, score=0.881 total time=   0.0s
[CV 3/5] END bootstrap=True, max_depth=50, max_features=10, n_estimators=10;, score=0.906 total time=   0.0s
[CV 4/5] END bootstrap=True, max_depth=50, max_features=

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished


### BERT + SVC

In [21]:
# Instantiate classifier
classifier = SVClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'C': [1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale']
    #'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}
classifier.randomized_search(X_train_embeddings_bert, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_bert, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_svc_bert_train, accuracy_svc_bert_train, f1_svc_bert_train = classifier.evaluate(X_train_embeddings_bert, y_train)

print('\n----- TEST -----')
cm_svc_bert_test, accuracy_svc_bert_test, f1_svc_bert_test = classifier.evaluate(X_test_embeddings_bert, y_test)
print('_______________________________________________________________________')

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ...C=1, gamma=scale, kernel=linear;, score=0.981 total time=   0.0s
[CV 2/5] END ...C=1, gamma=scale, kernel=linear;, score=0.975 total time=   0.0s
[CV 3/5] END ...C=1, gamma=scale, kernel=linear;, score=0.981 total time=   0.0s
[CV 4/5] END ...C=1, gamma=scale, kernel=linear;, score=0.975 total time=   0.0s
[CV 5/5] END ...C=1, gamma=scale, kernel=linear;, score=0.969 total time=   0.0s
[CV 1/5] END .....C=1, gamma=scale, kernel=poly;, score=0.975 total time=   0.0s
[CV 2/5] END .....C=1, gamma=scale, kernel=poly;, score=0.975 total time=   0.0s
[CV 3/5] END .....C=1, gamma=scale, kernel=poly;, score=0.969 total time=   0.0s
[CV 4/5] END .....C=1, gamma=scale, kernel=poly;, score=0.963 total time=   0.0s
[CV 5/5] END .....C=1, gamma=scale, kernel=poly;, score=0.956 total time=   0.0s
[CV 1/5] END ......C=1, gamma=scale, kernel=rbf;, score=0.975 total time=   0.0s
[CV 2/5] END ......C=1, gamma=scale, kernel=rbf;

### BERT + Logistic Regression

In [22]:
# Instantiate classifier
classifier = LRClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'C' : np.arange(0, 1, 0.01)
}
classifier.randomized_search(X_train_embeddings_bert, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_bert, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_lr_bert_train, accuracy_lr_bert_train, f1_lr_bert_train = classifier.evaluate(X_train_embeddings_bert, y_train)

print('\n----- TEST -----')
cm_lr_bert_test, accuracy_lr_bert_test, f1_lr_bert_test = classifier.evaluate(X_test_embeddings_bert, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END C=0.75, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.75, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.75, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.75, penalty=none, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.75, penalty=none, solver=liblinear;, score=nan total time=   0.0s


/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 1/5] END .C=0.76, penalty=none, solver=saga;, score=0.975 total time=   0.4s


/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 2/5] END .C=0.76, penalty=none, solver=saga;, score=0.988 total time=   0.4s


...

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 4/5] END ..C=0.28, penalty=none, solver=sag;, score=0.969 total time=   0.3s


/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 5/5] END ..C=0.28, penalty=none, solver=sag;, score=0.963 total time=   0.3s
[CV 1/5] END C=0.32, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 2/5] END C=0.32, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5] END C=0.32, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 4/5] END C=0.32, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 5/5] END C=0.32, penalty=l1, solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5] END C=0.84, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.84, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.84, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.84, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.84, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END C=0.29, penalty=l2, solver=liblinear;, score=0.981 total

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 464, in _check_

## GloVe embeddings

### GloVe + KNN

In [23]:
# Instantiate classifier
classifier = KNNClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9],
    'weights' : ['uniform','distance'],
    'metric' : ['minkowski','euclidean','manhattan']
}
classifier.randomized_search(X_train_embeddings_glove, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_glove, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_knn_glove_train, accuracy_knn_glove_train, f1_knn_glove_train = classifier.evaluate(X_train_embeddings_glove, y_train)

print('\n----- TEST -----')
cm_knn_glove_test, accuracy_knn_glove_test, f1_knn_glove_test = classifier.evaluate(X_test_embeddings_glove, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=manhattan, n_neighbors=5, weights=distance;, score=0.906 total time=   0.0s
[CV 2/5] END metric=manhattan, n_neighbors=5, weights=distance;, score=0.875 total time=   0.0s
[CV 3/5] END metric=manhattan, n_neighbors=5, weights=distance;, score=0.906 total time=   0.0s
[CV 4/5] END metric=manhattan, n_neighbors=5, weights=distance;, score=0.850 total time=   0.0s
[CV 5/5] END metric=manhattan, n_neighbors=5, weights=distance;, score=0.906 total time=   0.0s
[CV 1/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.919 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.875 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.881 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.869 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=9, weights=distance;, score=0.9

### GloVe + XGBoost

In [24]:
# Instantiate classifier
classifier = XGBoostClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'learning_rate': [0.01, 0.1, 0.3, 0.5],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7],
    'n_estimators': [50, 100, 200, 500],
    'objective': ['reg:squarederror']
}
classifier.randomized_search(X_train_embeddings_glove, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_glove, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_xgb_glove_train, accuracy_xgb_glove_train, f1_xgb_glove_train = classifier.evaluate(X_train_embeddings_glove, y_train)

print('\n----- TEST -----')
cm_xgb_glove_test, accuracy_xgb_glove_test, f1_xgb_glove_test = classifier.evaluate(X_test_embeddings_glove, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END colsample_bytree=0.7, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=50, objective=reg:squarederror, subsample=0.5;, score=0.875 total time=   0.2s
[CV 2/5] END colsample_bytree=0.7, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=50, objective=reg:squarederror, subsample=0.5;, score=0.881 total time=   0.1s
[CV 3/5] END colsample_bytree=0.7, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=50, objective=reg:squarederror, subsample=0.5;, score=0.869 total time=   0.1s
[CV 4/5] END colsample_bytree=0.7, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=50, objective=reg:squarederror, subsample=0.5;, score=0.894 total time=   0.1s
[CV 5/5] END colsample_bytree=0.7, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=50, objective=reg:squarederror, subsample=0.5;, score=0.887 total time=   0.1s
[CV 1/5] END colsample_bytree=0.5, learning

### GloVe + Random Forest

In [25]:
# Instantiate classifier
classifier = RFClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_estimators': [10, 25], 
    'max_features': [5, 10],
    'max_depth': [10, 50, None], 
    'bootstrap': [True, False]
}
classifier.randomized_search(X_train_embeddings_glove, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_glove, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_rf_glove_train, accuracy_rf_glove_train, f1_rf_glove_train = classifier.evaluate(X_train_embeddings_glove, y_train)

print('\n----- TEST -----')
cm_rf_glove_test, accuracy_rf_glove_test, f1_rf_glove_test = classifier.evaluate(X_test_embeddings_glove, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END bootstrap=True, max_depth=10, max_features=5, n_estimators=25;, score=0.856 total time=   0.1s
[CV 2/5] END bootstrap=True, max_depth=10, max_features=5, n_estimators=25;, score=0.863 total time=   0.1s
[CV 3/5] END bootstrap=True, max_depth=10, max_features=5, n_estimators=25;, score=0.881 total time=   0.1s
[CV 4/5] END bootstrap=True, max_depth=10, max_features=5, n_estimators=25;, score=0.931 total time=   0.1s
[CV 5/5] END bootstrap=True, max_depth=10, max_features=5, n_estimators=25;, score=0.925 total time=   0.0s
[CV 1/5] END bootstrap=False, max_depth=10, max_features=5, n_estimators=25;, score=0.887 total time=   0.1s
[CV 2/5] END bootstrap=False, max_depth=10, max_features=5, n_estimators=25;, score=0.875 total time=   0.1s
[CV 3/5] END bootstrap=False, max_depth=10, max_features=5, n_estimators=25;, score=0.869 total time=   0.1s
[CV 4/5] END bootstrap=False, max_depth=10, max_features=5, n_estimator

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished


### GloVe + SVC

In [26]:
# Instantiate classifier
classifier = SVClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'C': [1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale']
    #'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}
classifier.randomized_search(X_train_embeddings_glove, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_glove, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_svc_glove_train, accuracy_svc_glove_train, f1_svc_glove_train = classifier.evaluate(X_train_embeddings_glove, y_train)

print('\n----- TEST -----')
cm_svc_glove_test, accuracy_svc_glove_test, f1_svc_glove_test = classifier.evaluate(X_test_embeddings_glove, y_test)
print('_______________________________________________________________________')

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ...C=1, gamma=scale, kernel=linear;, score=0.931 total time=   0.0s
[CV 2/5] END ...C=1, gamma=scale, kernel=linear;, score=0.944 total time=   0.0s
[CV 3/5] END ...C=1, gamma=scale, kernel=linear;, score=0.975 total time=   0.0s
[CV 4/5] END ...C=1, gamma=scale, kernel=linear;, score=0.956 total time=   0.0s
[CV 5/5] END ...C=1, gamma=scale, kernel=linear;, score=0.969 total time=   0.0s
[CV 1/5] END .....C=1, gamma=scale, kernel=poly;, score=0.919 total time=   0.0s
[CV 2/5] END .....C=1, gamma=scale, kernel=poly;, score=0.919 total time=   0.0s
[CV 3/5] END .....C=1, gamma=scale, kernel=poly;, score=0.956 total time=   0.0s
[CV 4/5] END .....C=1, gamma=scale, kernel=poly;, score=0.931 total time=   0.0s
[CV 5/5] END .....C=1, gamma=scale, kernel=poly;, score=0.969 total time=   0.0s
[CV 1/5] END ......C=1, gamma=scale, kernel=rbf;, score=0.919 total time=   0.0s
[CV 2/5] END ......C=1, gamma=scale, kernel=rbf;

### GloVe + Logistic Regression

In [27]:
# Instantiate classifier
classifier = LRClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'C' : np.arange(0, 1, 0.01)
}
classifier.randomized_search(X_train_embeddings_glove, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_glove, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_lr_glove_train, accuracy_lr_glove_train, f1_lr_glove_train = classifier.evaluate(X_train_embeddings_glove, y_train)

print('\n----- TEST -----')
cm_lr_glove_test, accuracy_lr_glove_test, f1_lr_glove_test = classifier.evaluate(X_test_embeddings_glove, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END C=0.32, penalty=none, solver=newton-cg;, score=0.956 total time=   0.0s
[CV 2/5] END C=0.32, penalty=none, solver=newton-cg;, score=0.963 total time=   0.0s
[CV 3/5] END C=0.32, penalty=none, solver=newton-cg;, score=0.950 total time=   0.0s
[CV 4/5] END C=0.32, penalty=none, solver=newton-cg;, score=0.969 total time=   0.0s
[CV 5/5] END C=0.32, penalty=none, solver=newton-cg;, score=0.975 total time=   0.0s
[CV 1/5] END C=0.1, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5] END C=0.1, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 3/5] END C=0.1, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 4/5] END C=0.1, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 5/5] END C=0.1, penalty=elasticnet, solver=lbfgs;, score=nan total time=   0.0s
[CV 1/5] END C=0.86, penalty=l2, solver=newton-cg;, score=0.912 total time=   0.0s
[CV 2/5] E

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_lin

[CV 5/5] END C=0.86, penalty=l2, solver=newton-cg;, score=0.963 total time=   0.0s
[CV 1/5] END ......C=0.81, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.81, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END ......C=0.81, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END ......C=0.81, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END ......C=0.81, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.6, penalty=l1, solver=liblinear;, score=0.887 total time=   0.0s
[CV 2/5] END C=0.6, penalty=l1, solver=liblinear;, score=0.931 total time=   0.0s
[CV 3/5] END C=0.6, penalty=l1, solver=liblinear;, score=0.875 total time=   0.0s
[CV 4/5] END C=0.6, penalty=l1, solver=liblinear;, score=0.881 total time=   0.0s
[CV 5/5] END C=0.6, penalty=l1, solver=liblinear;, score=0.925 total time=   0.0s
[CV 1/5] END ....C=0.2, penalty=l2, solver=saga;, score=0.900 total time=   0.2s


/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END ....C=0.2, penalty=l2, solver=saga;, score=0.919 total time=   0.2s


...

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END ...C=0.43, penalty=l2, solver=saga;, score=0.912 total time=   0.2s
[CV 5/5] END ...C=0.43, penalty=l2, solver=saga;, score=0.956 total time=   0.2s
[CV 1/5] END ......C=0.74, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.74, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END ......C=0.74, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END ......C=0.74, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END ......C=0.74, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.7000000000000001, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END C=0.7000000000000001, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END C=0.7000000000000001, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END C=0.7000000000000001, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END C=0.7000000000000001, penalty=l1, solver=sag;, score=nan total t

/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/szymon/anaconda3/lib/python3.9

[CV 4/5] END C=0.35000000000000003, penalty=l2, solver=newton-cg;, score=0.912 total time=   0.1s
[CV 5/5] END C=0.35000000000000003, penalty=l2, solver=newton-cg;, score=0.950 total time=   0.0s
[CV 1/5] END C=0.48, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 2/5] END C=0.48, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 3/5] END C=0.48, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 4/5] END C=0.48, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 5/5] END C=0.48, penalty=elasticnet, solver=liblinear;, score=nan total time=   0.0s
[CV 1/5] END ......C=0.13, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.13, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END ......C=0.13, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END ......C=0.13, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END ......C=0.13, penalty=

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


## Word2Vec

### Word2Vec + KNN

In [28]:
# Instantiate classifier
classifier = KNNClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9],
    'weights' : ['uniform','distance'],
    'metric' : ['minkowski','euclidean','manhattan']
}
classifier.randomized_search(X_train_embeddings_word2vec, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_word2vec, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_knn_w2v_train, accuracy_knn_w2v_train, f1_knn_w2v_train = classifier.evaluate(X_train_embeddings_word2vec, y_train)

print('\n----- TEST -----')
cm_knn_w2v_test, accuracy_knn_w2v_test, f1_knn_w2v_test = classifier.evaluate(X_test_embeddings_word2vec, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END metric=minkowski, n_neighbors=2, weights=distance;, score=0.887 total time=   0.0s
[CV 2/5] END metric=minkowski, n_neighbors=2, weights=distance;, score=0.863 total time=   0.0s
[CV 3/5] END metric=minkowski, n_neighbors=2, weights=distance;, score=0.850 total time=   0.0s
[CV 4/5] END metric=minkowski, n_neighbors=2, weights=distance;, score=0.819 total time=   0.0s
[CV 5/5] END metric=minkowski, n_neighbors=2, weights=distance;, score=0.850 total time=   0.0s
[CV 1/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.900 total time=   0.0s
[CV 2/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.850 total time=   0.0s
[CV 3/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.887 total time=   0.0s
[CV 4/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.844 total time=   0.0s
[CV 5/5] END metric=euclidean, n_neighbors=3, weights=uniform;, score=0.906 to

### Word2Vec + XGBoost

In [29]:
# Instantiate classifier
classifier = XGBoostClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'learning_rate': [0.01, 0.1, 0.3, 0.5],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.5, 0.7],
    'colsample_bytree': [0.5, 0.7],
    'n_estimators': [50, 100, 200, 500],
    'objective': ['reg:squarederror']
}
classifier.randomized_search(X_train_embeddings_word2vec, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_word2vec, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_xgb_w2v_train, accuracy_xgb_w2v_train, f1_xgb_w2v_train = classifier.evaluate(X_train_embeddings_word2vec, y_train)

print('\n----- TEST -----')
cm_xgb_w2v_test, accuracy_xgb_w2v_test, f1_xgb_w2v_test = classifier.evaluate(X_test_embeddings_word2vec, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END colsample_bytree=0.5, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.5;, score=0.906 total time=   0.8s
[CV 2/5] END colsample_bytree=0.5, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.5;, score=0.950 total time=   0.9s
[CV 3/5] END colsample_bytree=0.5, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.5;, score=0.912 total time=   0.9s
[CV 4/5] END colsample_bytree=0.5, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.5;, score=0.925 total time=   0.8s
[CV 5/5] END colsample_bytree=0.5, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=500, objective=reg:squarederror, subsample=0.5;, score=0.950 total time=   0.8s
[CV 1/5] END colsample_bytree=0.5

### Word2Vec + Random Forest

In [30]:
# Instantiate classifier
classifier = RFClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'n_estimators': [10, 25],
    'max_features': [5, 10],
    'max_depth': [10, 50, None],
    'bootstrap': [True, False]
}
classifier.randomized_search(X_train_embeddings_word2vec, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_word2vec, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_rf_w2v_train, accuracy_rf_w2v_train, f1_rf_w2v_train = classifier.evaluate(X_train_embeddings_word2vec, y_train)

print('\n----- TEST -----')
cm_rf_w2v_test, accuracy_rf_w2v_test, f1_rf_w2v_test = classifier.evaluate(X_test_embeddings_word2vec, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END bootstrap=True, max_depth=None, max_features=5, n_estimators=25;, score=0.887 total time=   0.1s
[CV 2/5] END bootstrap=True, max_depth=None, max_features=5, n_estimators=25;, score=0.900 total time=   0.1s
[CV 3/5] END bootstrap=True, max_depth=None, max_features=5, n_estimators=25;, score=0.906 total time=   0.1s
[CV 4/5] END bootstrap=True, max_depth=None, max_features=5, n_estimators=25;, score=0.919 total time=   0.0s
[CV 5/5] END bootstrap=True, max_depth=None, max_features=5, n_estimators=25;, score=0.944 total time=   0.0s
[CV 1/5] END bootstrap=False, max_depth=50, max_features=5, n_estimators=25;, score=0.906 total time=   0.1s
[CV 2/5] END bootstrap=False, max_depth=50, max_features=5, n_estimators=25;, score=0.919 total time=   0.1s
[CV 3/5] END bootstrap=False, max_depth=50, max_features=5, n_estimators=25;, score=0.900 total time=   0.1s
[CV 4/5] END bootstrap=False, max_depth=50, max_features=5, n

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.0s finished


### Word2Vec + SVC

In [31]:
# Instantiate classifier
classifier = SVClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'C': [1, 10, 100, 1000],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale']
    #'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}
classifier.randomized_search(X_train_embeddings_word2vec, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_word2vec, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_svc_w2v_train, accuracy_svc_w2v_train, f1_svc_w2v_train = classifier.evaluate(X_train_embeddings_word2vec, y_train)

print('\n----- TEST -----')
cm_svc_w2v_test, accuracy_svc_w2v_test, f1_svc_w2v_test = classifier.evaluate(X_test_embeddings_word2vec, y_test)
print('_______________________________________________________________________')

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ...C=1, gamma=scale, kernel=linear;, score=0.938 total time=   0.0s
[CV 2/5] END ...C=1, gamma=scale, kernel=linear;, score=0.938 total time=   0.0s
[CV 3/5] END ...C=1, gamma=scale, kernel=linear;, score=0.938 total time=   0.0s
[CV 4/5] END ...C=1, gamma=scale, kernel=linear;, score=0.950 total time=   0.0s
[CV 5/5] END ...C=1, gamma=scale, kernel=linear;, score=0.963 total time=   0.0s
[CV 1/5] END .....C=1, gamma=scale, kernel=poly;, score=0.938 total time=   0.0s
[CV 2/5] END .....C=1, gamma=scale, kernel=poly;, score=0.938 total time=   0.0s
[CV 3/5] END .....C=1, gamma=scale, kernel=poly;, score=0.931 total time=   0.0s
[CV 4/5] END .....C=1, gamma=scale, kernel=poly;, score=0.950 total time=   0.0s
[CV 5/5] END .....C=1, gamma=scale, kernel=poly;, score=0.975 total time=   0.0s
[CV 1/5] END ......C=1, gamma=scale, kernel=rbf;, score=0.944 total time=   0.0s
[CV 2/5] END ......C=1, gamma=scale, kernel=rbf;

### Word2Vec + Logistic Regression

In [32]:
# Instantiate classifier
classifier = LRClassifier()

# Perform randomized search over hyperparameters
param_distributions = {
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'C' : np.arange(0, 1, 0.01)
}
classifier.randomized_search(X_train_embeddings_word2vec, y_train, param_distributions)

# Train classifier on training data
classifier.fit(X_train_embeddings_word2vec, y_train)

# Evaluate classifier
print('\n')
print('\n_______________________________________________________________________')
print('EVALUATION')

print('\n----- TRAIN -----')
cm_lr_w2v_train, accuracy_lr_w2v_train, f1_lr_w2v_train = classifier.evaluate(X_train_embeddings_word2vec, y_train)

print('\n----- TEST -----')
cm_lr_w2v_test, accuracy_lr_w2v_test, f1_lr_w2v_test = classifier.evaluate(X_test_embeddings_word2vec, y_test)
print('_______________________________________________________________________')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END C=0.8200000000000001, penalty=l2, solver=saga;, score=0.938 total time=   0.1s
[CV 2/5] END C=0.8200000000000001, penalty=l2, solver=saga;, score=0.925 total time=   0.1s
[CV 3/5] END C=0.8200000000000001, penalty=l2, solver=saga;, score=0.906 total time=   0.1s
[CV 4/5] END C=0.8200000000000001, penalty=l2, solver=saga;, score=0.919 total time=   0.1s
[CV 5/5] END C=0.8200000000000001, penalty=l2, solver=saga;, score=0.956 total time=   0.0s
[CV 1/5] END C=0.41000000000000003, penalty=l1, solver=liblinear;, score=0.850 total time=   0.0s
[CV 2/5] END C=0.41000000000000003, penalty=l1, solver=liblinear;, score=0.894 total time=   0.0s
[CV 3/5] END C=0.41000000000000003, penalty=l1, solver=liblinear;, score=0.894 total time=   0.0s
[CV 4/5] END C=0.41000000000000003, penalty=l1, solver=liblinear;, score=0.900 total time=   0.0s
[CV 5/5] END C=0.41000000000000003, penalty=l1, solver=liblinear;, score=0.900 total t

/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/home/szymon/anaconda3/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


[CV 4/5] END C=0.07, penalty=l2, solver=newton-cg;, score=0.887 total time=   0.1s
[CV 5/5] END C=0.07, penalty=l2, solver=newton-cg;, score=0.925 total time=   0.0s


/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/5] END ...C=0.29, penalty=l1, solver=saga;, score=0.838 total time=   0.2s


...

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 5/5] END ...C=0.29, penalty=l1, solver=saga;, score=0.894 total time=   0.2s
[CV 1/5] END ......C=0.67, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END ......C=0.67, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END ......C=0.67, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END ......C=0.67, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END ......C=0.67, penalty=l1, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.49, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END C=0.49, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END C=0.49, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END C=0.49, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END C=0.49, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END ....C=0.33, penalty=l2, solver=sag;, score=0.925 total time=   0.0s
[CV 2/5] END ....C

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(


[CV 4/5] END ...C=0.55, penalty=l2, solver=saga;, score=0.919 total time=   0.0s
[CV 5/5] END ...C=0.55, penalty=l2, solver=saga;, score=0.938 total time=   0.0s
[CV 1/5] END .......C=0.0, penalty=l2, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END .......C=0.0, penalty=l2, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END .......C=0.0, penalty=l2, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END .......C=0.0, penalty=l2, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END .......C=0.0, penalty=l2, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.59, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 2/5] END C=0.59, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 3/5] END C=0.59, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 4/5] END C=0.59, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 5/5] END C=0.59, penalty=elasticnet, solver=sag;, score=nan total time=   0.0s
[CV 1/5] END C=0.5

/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:858: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = 1.0 / C
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:320: RuntimeWarning: invalid value encountered in double_scalars
  if step_size * alpha_scaled == 1:
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:858: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = 1.0 / C
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:320: RuntimeWarning: invalid value encountered in double_scalars
  if step_size * alpha_scaled == 1:
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:858: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = 1.0 / C
/home/szymon/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:320: RuntimeWarning: invalid value encountered in d

# Results

In [33]:
#create data
data = [["KNN + BERT", accuracy_knn_bert_test, f1_knn_bert_test], 
        ["XGBoost + BERT", accuracy_xgb_bert_test, f1_xgb_bert_test], 
        ["Random Forest + BERT", accuracy_rf_bert_test, f1_rf_bert_test],
        ["SVC + BERT", accuracy_svc_bert_test, f1_svc_bert_test], 
        ["LR + BERT", accuracy_lr_bert_test, f1_lr_bert_test],
        ["KNN + GloVe", accuracy_knn_glove_test, f1_knn_glove_test],         
        ["XGBoost + GloVe", accuracy_xgb_glove_test, f1_xgb_glove_test],
        ["Random Forest + GloVe", accuracy_rf_glove_test, f1_rf_glove_test],
        ["SVC + GloVe", accuracy_svc_glove_test, f1_svc_glove_test],
        ["LR + GloVe", accuracy_lr_glove_test, f1_lr_glove_test],
        ["KNN + Word2Vec", accuracy_knn_w2v_test, f1_knn_w2v_test],
        ["XGBoost + Word2Vec", accuracy_xgb_w2v_test, f1_xgb_w2v_test],
        ["Random Forest + Word2Vec", accuracy_rf_w2v_test, f1_rf_w2v_test],
        ["SVC + Word2Vec", accuracy_svc_w2v_test, f1_svc_w2v_test],
        ["LR + Word2Vec", accuracy_lr_w2v_test, f1_lr_w2v_test]]
  
#define header names
col_names = ["Model", "Accuracy", "F1-Score"]
  
#display table
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))

╒══════════════════════════╤════════════╤════════════╕
│ Model                    │   Accuracy │   F1-Score │
╞══════════════════════════╪════════════╪════════════╡
│ KNN + BERT               │       96.5 │       96.5 │
├──────────────────────────┼────────────┼────────────┤
│ XGBoost + BERT           │       95   │       95   │
├──────────────────────────┼────────────┼────────────┤
│ Random Forest + BERT     │       94   │       94   │
├──────────────────────────┼────────────┼────────────┤
│ SVC + BERT               │       98.5 │       98.5 │
├──────────────────────────┼────────────┼────────────┤
│ LR + BERT                │       98   │       98   │
├──────────────────────────┼────────────┼────────────┤
│ KNN + GloVe              │       89.5 │       89.4 │
├──────────────────────────┼────────────┼────────────┤
│ XGBoost + GloVe          │       92.5 │       92.5 │
├──────────────────────────┼────────────┼────────────┤
│ Random Forest + GloVe    │       87.5 │       87.4 │
├─────────

In [34]:
#create data
knn_results = [["BERT", accuracy_knn_bert_test, f1_knn_bert_test],
        ["GloVe", accuracy_knn_glove_test, f1_knn_glove_test],
        ["Word2Vec", accuracy_knn_w2v_test, f1_knn_w2v_test]]

#define header names
col_names = ["Model", "Accuracy", "F1-Score"]

#display table
print("KNN")
print(tabulate(knn_results, headers=col_names, tablefmt="fancy_grid"))

KNN
╒══════════╤════════════╤════════════╕
│ Model    │   Accuracy │   F1-Score │
╞══════════╪════════════╪════════════╡
│ BERT     │       96.5 │       96.5 │
├──────────┼────────────┼────────────┤
│ GloVe    │       89.5 │       89.4 │
├──────────┼────────────┼────────────┤
│ Word2Vec │       91.5 │       91.4 │
╘══════════╧════════════╧════════════╛


In [35]:
#create data
xgb_results = [["BERT", accuracy_xgb_bert_test, f1_xgb_bert_test],
        ["GloVe", accuracy_xgb_glove_test, f1_xgb_glove_test],
        ["Word2Vec", accuracy_xgb_w2v_test, f1_xgb_w2v_test]]

#define header names
col_names = ["Model", "Accuracy", "F1-Score"]

#display table
print("XGBoost")
print(tabulate(xgb_results, headers=col_names, tablefmt="fancy_grid"))

XGBoost
╒══════════╤════════════╤════════════╕
│ Model    │   Accuracy │   F1-Score │
╞══════════╪════════════╪════════════╡
│ BERT     │       95   │       95   │
├──────────┼────────────┼────────────┤
│ GloVe    │       92.5 │       92.5 │
├──────────┼────────────┼────────────┤
│ Word2Vec │       96   │       96   │
╘══════════╧════════════╧════════════╛


In [36]:
#create data
rf_results = [["BERT", accuracy_rf_bert_test, f1_rf_bert_test],
        ["GloVe", accuracy_rf_glove_test, f1_rf_glove_test],
        ["Word2Vec", accuracy_rf_w2v_test, f1_rf_w2v_test]]

#define header names
col_names = ["Model", "Accuracy", "F1-Score"]

#display table
print("Random Forest")
print(tabulate(rf_results, headers=col_names, tablefmt="fancy_grid"))

Random Forest
╒══════════╤════════════╤════════════╕
│ Model    │   Accuracy │   F1-Score │
╞══════════╪════════════╪════════════╡
│ BERT     │       94   │       94   │
├──────────┼────────────┼────────────┤
│ GloVe    │       87.5 │       87.4 │
├──────────┼────────────┼────────────┤
│ Word2Vec │       91   │       91.1 │
╘══════════╧════════════╧════════════╛


In [37]:
#create data
svc_results = [["BERT", accuracy_svc_bert_test, f1_svc_bert_test],
        ["GloVe", accuracy_svc_glove_test, f1_svc_glove_test],
        ["Word2Vec", accuracy_svc_w2v_test, f1_svc_w2v_test]]

#define header names
col_names = ["Model", "Accuracy", "F1-Score"]

#display table
print("SVM Classifier")
print(tabulate(svc_results, headers=col_names, tablefmt="fancy_grid"))

SVM Classifier
╒══════════╤════════════╤════════════╕
│ Model    │   Accuracy │   F1-Score │
╞══════════╪════════════╪════════════╡
│ BERT     │       98.5 │       98.5 │
├──────────┼────────────┼────────────┤
│ GloVe    │       94   │       93.9 │
├──────────┼────────────┼────────────┤
│ Word2Vec │       97   │       96.9 │
╘══════════╧════════════╧════════════╛


In [38]:
#create data
lr_results = [["BERT", accuracy_lr_bert_test, f1_lr_bert_test],
        ["GloVe", accuracy_lr_glove_test, f1_lr_glove_test],
        ["Word2Vec", accuracy_lr_w2v_test, f1_lr_w2v_test]]

#define header names
col_names = ["Model", "Accuracy", "F1-Score"]

#display table
print("Logistic Regression")
print(tabulate(lr_results, headers=col_names, tablefmt="fancy_grid"))

Logistic Regression
╒══════════╤════════════╤════════════╕
│ Model    │   Accuracy │   F1-Score │
╞══════════╪════════════╪════════════╡
│ BERT     │       98   │       98   │
├──────────┼────────────┼────────────┤
│ GloVe    │       94.5 │       94.4 │
├──────────┼────────────┼────────────┤
│ Word2Vec │       95.5 │       95.5 │
╘══════════╧════════════╧════════════╛


---